<a href="https://colab.research.google.com/github/1ZeroStorm/Machine_Learning_Projects/blob/main/DecisionTree/DecisionTree_FromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import tools
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
import statistics
import inspect

In [ ]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns = iris.feature_names)


In [ ]:
X = iris.data
y = iris.target.reshape(-1, 1) # 1d -> 2d array
full_dataset = np.concatenate([X, y], axis = 1)
full_dataset[:5]

array([[5.1, 3.5, 1.4, 0.2, 0. ],
       [4.9, 3. , 1.4, 0.2, 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. ],
       [5. , 3.6, 1.4, 0.2, 0. ]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print(X_train.shape[0])
print(X_test.shape)
print(X.shape)

120
(30, 4)
(150, 4)


In [ ]:
full_train_dataset = np.concatenate([X_train, y_train], axis = 1)
full_train_dataset[:5]

array([[6. , 2.9, 4.5, 1.5, 1. ],
       [4.9, 3.1, 1.5, 0.2, 0. ],
       [5.3, 3.7, 1.5, 0.2, 0. ],
       [6.7, 3. , 5.2, 2.3, 2. ],
       [6.7, 3.1, 5.6, 2.4, 2. ]])

In [ ]:
class Node():
  def __init__(self, leftChild = None, rightChild  = None, threshold = None, curr_depth = None, feature_index  = None, value = None):
    self.leftChild = leftChild
    self.rightChild = rightChild
    self.threshold= threshold
    self.curr_depth = curr_depth
    self.feature_index = feature_index
    self.value = value

In [ ]:
class DecisionTreeClassifier():
  def __init__(self, min_samples_split, max_depth) -> None:
    self.min_samples_split =  min_samples_split
    self.max_depth = max_depth
    self.root = None

  def fit(self, original_dataset):
    self.root = self.createTree(original_dataset)

  def createTree(self, passed_dataset, currdepth = 0):
    XX = passed_dataset[:, :-1]
    yy = passed_dataset[:, -1]

    num_rows, num_cols = XX.shape

    if num_rows >= self.min_samples_split and currdepth <= self.max_depth:
      getBestsplitfrom_parent = self.get_best_split(XX, yy, passed_dataset)
      #print(getBestsplitfrom_parent['leftChild'])
      if getBestsplitfrom_parent['info_gain'] > 0:

        leftChild = self.createTree(getBestsplitfrom_parent['leftChild'], currdepth + 1)
        rightChild = self.createTree(getBestsplitfrom_parent['rightChild'], currdepth + 1)
        threshold = getBestsplitfrom_parent['threshold']
        col_index = getBestsplitfrom_parent['col_index']
        return Node(leftChild, rightChild, threshold, currdepth, col_index)

    mode_value = statistics.mode(yy.tolist())
    return Node(value = mode_value)

  def get_best_split(self, XX, yy, full_dataset):
    num_rows, num_cols = XX.shape

    max_information_gain = -float('inf')
    best_split = {}

    for row_index in range(num_rows):
      curr_row = XX[row_index]
      for col_index in range(num_cols):
        curr_thres = curr_row[col_index]
        left_rows = np.array([row.tolist() for row in full_dataset if row[col_index]<=curr_thres]) # 2d array
        right_rows = np.array([row.tolist() for row in full_dataset if row[col_index]>curr_thres])

        if len(left_rows) > 0 and len(right_rows) > 0:
          y_left = left_rows[:, -1]
          y_right = right_rows[:, -1]
          y_full = np.concatenate((y_left, y_right))

          new_information_gain = self.get_information_gain(y_left, y_right, y_full, mode = 'gini')

          if new_information_gain > max_information_gain:
            best_split['leftChild'] = left_rows
            best_split['rightChild'] = right_rows
            best_split['parent'] =  full_dataset
            best_split['threshold'] = curr_thres
            best_split['col_index'] = col_index

            max_information_gain = new_information_gain

    best_split['info_gain'] = max_information_gain
    #print(best_split['leftChild'], '\n', best_split['rightChild'], '\n', best_split['parent'])
    return best_split


  def get_information_gain(self,y_left, y_right, y_full, mode='gini'):

    weight_left = len(y_left) / len(y_full)
    weight_right = len(y_right)/ len(y_full)

    if mode == 'gini':
      gp = self.gini_index(y_full)
      gl_c = self.gini_index(y_left)
      gr_c = self.gini_index(y_right)

      gain = gp - (weight_left * gl_c + weight_right * gr_c)
      return gain

    elif mode == 'entropy':
      Eparent = self.entropy(y_full)
      El_c = self.entropy(y_left)
      Er_c = self.entropy(y_right)

      gain = Eparent - (weight_left * El_c + weight_right * Er_c)
      return gain

  def gini_index(self, one_d_data):
    gini = 0
    unique = np.unique(one_d_data)
    for value in unique:
      p_class = len([element for element in one_d_data if element == value]) / len(one_d_data)
      gini+= p_class ** 2

    return 1-gini

  def entropy(self, one_d_data):
    entropy = 0

    unique = np.unique(one_d_data)
    for value in unique:
      p_class = len([element for element in one_d_data if element == value]) / len(one_d_data)
      formula = - p_class * math.log2(p_class)
      entropy += formula
    return entropy

  def printTree(self, tree=None, depth=0, ):
    """Recursively print the tree structure"""

    if tree is None:
        tree = self.root

    indent = "  " * depth  # 2 spaces per depth level

    if tree.value is None:
      # Print node information
      print(f"{indent}feature={tree.feature_index}, threshold={tree.threshold:.3f}")

      # Recursively print left child
      if tree.leftChild:
          self.printTree(tree.leftChild, depth + 1)

      # Recursively print right child
      if tree.rightChild:
          self.printTree(tree.rightChild, depth + 1)

    else:  # Leaf node
        print(f"{indent} leaf: {tree.value}")

  def predict(self, X):
    """Predict class for each sample in X"""
    predictions = []
    for sample in X:
        pred = self._predict_single(sample, self.root)
        predictions.append(pred)
    return np.array(predictions)

  def _predict_single(self, x, node):
    """Recursively traverse tree for a single sample"""
    if node.value is not None: return node.value

    # If it's a Node object, check threshold
    if x[node.feature_index] <= node.threshold:
        return self._predict_single(x, node.leftChild)
    else:
        return self._predict_single(x, node.rightChild)




In [ ]:
newDTC = DecisionTreeClassifier(3, 3)
newDTC.fit(full_train_dataset)

newDTC.printTree()


feature=2, threshold=1.900
   leaf: 0.0
  feature=3, threshold=1.700
    feature=2, threshold=5.100
      feature=0, threshold=4.900
         leaf: 1.0
         leaf: 1.0
       leaf: 2.0
    feature=2, threshold=4.800
      feature=0, threshold=5.900
         leaf: 1.0
         leaf: 2.0
       leaf: 2.0


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Get predictions
y_pred = newDTC.predict(X_test)
y_pred

array([1., 1., 2., 2., 1., 0., 1., 1., 1., 0., 2., 1., 0., 0., 1., 2., 0.,
       2., 1., 2., 0., 1., 2., 0., 0., 1., 0., 1., 1., 2.])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9333333333333333